In [ ]:
from joblib import Parallel, delayed
import gzip,os,tarfile,sys
sys.path.append(os.pardir+'/src')
from settings import *
import pandas as pd
from pandas import DataFrame
import time
from janome.tokenizer import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import time
%matplotlib inline 

In [ ]:
#　かなりの改良の余地あり
def cleanBody(x):
    try:
        hoge = x.split('（ブルームバーグ）：')
        if len(hoge)>1:
            temp = ''
            for i in range(1,len(hoge)):
                temp += hoge[i]
            y1 = temp
        else :
            y1 = x
    
        hoge = y1.split('(ブルームバーグ) -- ')
        if len(hoge)>1:
            temp = ''
            for i in range(1,len(hoge)):
                temp += hoge[i]
            y2 = temp
        else :
            y2 = y1
    
        hoge = y2.split('関連ニュースと情報：')
        if len(hoge)>1:
            temp = ''
            for i in range(0,len(hoge)-1):
                temp += hoge[i]
            y3 = temp
        else :
            y3 = y2
        
            hoge = y3.split('原題：')
        if len(hoge)>1:
            temp = ''
            for i in range(0,len(hoge)-1):
                temp += hoge[i]
            y4 = temp
        else :
            y4 = y3
        
    
            hoge = y3.split('参考画面：')
        if len(hoge)>1:
            temp = ''
            for i in range(0,len(hoge)-1):
                temp += hoge[i]
            y5 = temp
        else :
            y5 = y4
        
    
        y5 = y5.replace('\n', '')
        y5 = y5.replace(' ', '')
        y=y5
       
        return y

    except: 
        print(x)
        return x
    

In [ ]:
# need to recieve unicode text
def myTokenizer(text):
    TARGET_CATEGORY = ["名詞", "動詞",  "形容詞", "副詞"]
    wordsIn=[]
    t = Tokenizer()
    tokens = t.tokenize(text)
    for token in tokens:
        tokenCategory = token.part_of_speech.split(',')[0]
        tokenBasic = token.base_form
        if  (tokenCategory=='名詞' and token.part_of_speech.split(',')[1]=='固有名詞'):
            wordsIn.append(token.surface)
        elif tokenCategory in TARGET_CATEGORY:
            if tokenBasic != '*':                               #if basic form can be defined
                wordsIn.append(tokenBasic)
    return wordsIn

In [ ]:
def myCounter(x):
    global FOR_UNIQUE
    try:
        vectorizer = CountVectorizer(tokenizer=myTokenizer)
        bow_sparse = vectorizer.fit_transform(x)
        bow = bow_sparse.todense()
        return (np.sum(bow_sparse), bow.shape[1]) 
    except:
        #print('えらー')
        #print(x[0])
        #raise Exception
        return (-1, -1)            # なぞのエラー発生？ 
    

In [ ]:
files_2017 = os.listdir(DATA_DIR+'/EID34151/myCsv/2017')

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR,'EID34151/myCsv/2017/EID34151_20170101.csv'))
df = df.drop_duplicates(['Body'])
df = df.fillna('') 
df.Body = df['Body'].apply(lambda x: cleanBody(x))

In [ ]:
# case1
start = time.time()
hoge1 = df['Body'].apply(lambda x: myCounter([x]))
print('it took ', time.time() - start)

In [ ]:
# 繰り返し計算 (並列化)
if __name__ == '__main__':
    start = time.time()
    hoge2 = Parallel(n_jobs=-1)( [delayed(myCounter)(df.Body[i:i+1]) for i in range(4)] )
    print('it took ', time.time() - start)

In [ ]:
df = pd.read_csv(os.path.join(DATA_DIR,'EID34151/myCsv/2017/EID34151_01012017.csv'))
            df = df.drop_duplicates(['Body'])
            df = df.fillna('') 
            df.Body = df['Body'].apply(lambda x: cleanBody(x))
        
            start = time.time()

            df['NumOfTaxReduction'] = df['Body'].apply(lambda x: len(x.split('減税'))-1)
            df['NumOfTaxIncrease'] = df['Body'].apply(lambda x: len(x.split('増税'))-1)
            # エラーが発生するときは-1
            # janomeのせい？　mecab使えば対処できるかも
            df['NumOfWords'] = df['Body'].apply(lambda x: myCounter([x]))
            df['NumOfWords_Uni'] = pd.Series(getFOR_UNIQUE())
            df['NumOfSentences'] = df['Body'].apply(lambda x: len(x.split('。'))-1)

            df.to_csv(os.path.join(DATA_DIR,'EID34151/',file.replace(".xml.gz",".csv")),encoding='utf8',header=True,index=False)       
            clearFOR_UNIQUE()
            
            print(file,' took ', time.time() - start)